```
# Projeto de Detecção de Anomalias em Terminais de Loterias'**
```

# Os que têm nome de robô futurista

1.   Willyam Rodrigo
2.   Rafael Masashi
3.   Macario Passalini
4.   Matheus Boechat
5.   Lincon Agrizzi
6.   Gabriel Henrique




Upload DataSet: dados_loterias.csv

In [1]:
from google.colab import files
uploaded = files.upload()

Saving dados_loterias.csv to dados_loterias.csv


1. **Carregar e Pré-processar os Dados**

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

# --- 1. Carregar e Pré-processar os Dados (Corrigido) ---
# A função parse_dates já converte a coluna 'Timestamp'.
df = pd.read_csv('dados_loterias.csv', parse_dates=['Timestamp'])

# Converte as colunas para o tipo numérico, garantindo que não haja problemas
df['Num_Transacoes'] = pd.to_numeric(df['Num_Transacoes'], errors='coerce')
df['Valor_Total'] = pd.to_numeric(df['Valor_Total'], errors='coerce')

# Ordena os dados por terminal e por data/hora
df = df.sort_values(by=['ID_Terminal', 'Timestamp'])

# Codifica as colunas de texto para números.
df = pd.get_dummies(df, columns=['Status_Conexao', 'Status_Alarme'], dtype=int)

# --- Precisei excluir a coluna para antecipar futuros erros. AQUI ESTÁ A CORREÇÃO: Remova a coluna 'Timestamp' ---
df = df.drop(columns=['Timestamp'])

# Verifica os tipos de dados para confirmar a conversão
print(df.dtypes)
print("Passo 1: Pré-processamento concluído.")


ID_Terminal                 object
Num_Transacoes               int64
Valor_Total                float64
Status_Conexao_Ativo         int64
Status_Conexao_Falha         int64
Status_Alarme_Disparado      int64
Status_Alarme_Normal         int64
dtype: object
Passo 1: Pré-processamento concluído.


2. **Função para Criar as Sequências**

In [3]:
# --- 2. Função para Criar as Sequências (Sem Alterações) ---
def create_sequences(data, n_steps):
    X, y = [], []
    for terminal_id in data['ID_Terminal'].unique():
        # A coluna ID_Terminal é removida aqui antes da conversão para float32.
        terminal_data = data[data['ID_Terminal'] == terminal_id].drop(columns=['ID_Terminal']).copy()

        # O modelo LSTM requer valores float32
        terminal_data = terminal_data.astype('float32')

        for i in range(len(terminal_data) - n_steps):
            sequence = terminal_data.iloc[i:(i + n_steps)].values
            target = terminal_data['Num_Transacoes'].iloc[i + n_steps]

            X.append(sequence)
            y.append(target)

    return np.array(X), np.array(y)

3. **Criação das Sequências a partir do DataFrame numérico**

In [4]:
# --- 3. Criação das Sequências a partir do DataFrame numérico ---
n_steps = 3
X, y = create_sequences(df, n_steps)

print(f"Passo 2: Sequências criadas. X.shape: {X.shape}, y.shape: {y.shape}")

Passo 2: Sequências criadas. X.shape: (1352, 3, 6), y.shape: (1352,)


4. **Construção e Treinamento do Modelo LSTM**

In [5]:
# --- 4. Construção e Treinamento do Modelo LSTM ---
# As dimensões de entrada para o modelo LSTM são obtidas de X
num_passos_de_tempo = X.shape[1]
num_features = X.shape[2]

**Cria Arquitetura do Modelo**

In [6]:
# Cria o modelo
model = Sequential()
# Adiciona a camada de entrada
model.add(Input(shape=(num_passos_de_tempo, num_features)))
# Adiciona a camada LSTM
model.add(LSTM(10, activation='relu'))
# Adiciona a camada de saída
model.add(Dense(1))
# Exibe o resumo da arquitetura
model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 10)             │           680 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │            11 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 691 (2.70 KB)

 Trainable params: 691 (2.70 KB)

 Non-trainable params: 0 (0.00 B)

**Compila o modelo**

In [7]:
# Compila o modelo
model.compile(optimizer='adam', loss='mean_squared_error')

**Treina o modelo**

In [8]:
# Treina o modelo.
print("\nPasso 3: Treinando o modelo...")
model.fit(X, y, epochs=10, batch_size=32, verbose=1)
print("\nTreinamento do modelo concluído com sucesso! 😎")


Passo 3: Treinando o modelo...
Epoch 1/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 29467.8379
Epoch 2/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 9711.9121
Epoch 3/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4735.7119
Epoch 4/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4344.3398
Epoch 5/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3973.0059
Epoch 6/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3714.8513
Epoch 7/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 4477.0396
Epoch 8/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3692.3108
Epoch 9/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 3409.6729
Epoch 10/10
43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 3117.4121

Treinamento do modelo concluído com sucesso! 😎


# **`----- Detectar Anomalias -----`**

1. **Previsões usando o modelo treinado**

In [9]:
# Fazendo as previsões usando o modelo treinado
previsoes = model.predict(X)

print("Previsões geradas com sucesso.")

43/43 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
Previsões geradas com sucesso.




```
# Isto está formatado como código
```

2. ***Calculando o erro de prevsão***

In [10]:
import numpy as np

# Acha o erro (diferença) entre o valor real e a previsão
erros = np.mean(np.square(previsoes - y), axis=1)

print("Erros de previsão calculados.")

Erros de previsão calculados.


3. **Definindo o limiar de Anomalias e identificar Anomalias**

In [11]:
# Calcula a média e o desvio padrão dos erros
limiar_erro = np.mean(erros) + 3 * np.std(erros)

# Identifica os índices das sequências que têm um erro acima do limiar
indices_anomalias = np.where(erros > limiar_erro)[0]

print(f"Limiar de anomalia definido: {limiar_erro:.2f}")
print(f"Número de anomalias detectadas: {len(indices_anomalias)}")

Limiar de anomalia definido: 21154.08
Número de anomalias detectadas: 3


4. **Exibindo os Dados de Anomalia**

In [12]:
# Cria uma cópia do DataFrame original para análise
df_analise = df.copy()

# Cria a nova coluna 'Erro_Previsao'
df_analise['Erro_Previsao'] = np.nan

# Usa o método .loc para atribuir os valores de erro, e preenche apenas os valores dos índices detectados.
df_analise.loc[df_analise.index[indices_anomalias], 'Erro_Previsao'] = erros[indices_anomalias]

# Cria a coluna final para identificar as anomalias com 1.
df_analise['Anomalia_Prevista'] = (~df_analise['Erro_Previsao'].isna()).astype(int)

# Filtra e mostra os resultados
anomalias_detectadas = df_analise.loc[df_analise['Anomalia_Prevista'] == 1]
print("Anomalias detectadas pela IA:\n")
print(anomalias_detectadas)


Anomalias detectadas pela IA:

    ID_Terminal  Num_Transacoes  Valor_Total  Status_Conexao_Ativo  \
506       T-011              61   430.863568                     1   
911       T-011              83   521.861171                     1   
736       T-011              94   772.287799                     1   

     Status_Conexao_Falha  Status_Alarme_Disparado  Status_Alarme_Normal  \
506                     0                        0                     1   
911                     0                        0                     1   
736                     0                        0                     1   

     Erro_Previsao  Anomalia_Prevista  
506   34886.007812                  1  
911  158579.593750                  1  
736  147795.921875                  1  
